In [2]:
import pickle
import sys
import numpy as np
import pandas as pd
from fractions import Fraction
import re
import math

## PATHS

In [3]:
# inputs
ing_in_input = 'data/join/ing_in_table.pkl'
usda_input = 'data/join/usda.pkl'
recipes_input = 'data/raw/full_recipes.pkl'
# outputs
ing_in_output = 'data/tables/ing_in.xlsx'
recipes_output = 'data/tables/recipes.xlsx'
usda_output = 'data/tables/usda.xlsx'

## Load Inputs

In [4]:
def load(path):
    return pickle.load(open(path, 'rb'))

ing_in = load(ing_in_input)
usda = load(usda_input)
recipes = load(recipes_input)

Remove all usda ingredients from usda that do not have any matches in ingredients in. Drops about 3k ingredients

In [5]:
feat_ings = ing_in['usda_id']
feated = usda.index.isin(feat_ings.values)
usda = usda.loc[feated, :]

Drop unnecessary nutrition columns

In [6]:
usda.drop(columns=['Shrt_Desc', 'Water_(g)', 'Lipid_Tot_(g)',
       'Ash_(g)', 'Carbohydrt_(g)', 'Fiber_TD_(g)',
       'Calcium_(mg)', 'Iron_(mg)', 'Magnesium_(mg)', 'Phosphorus_(mg)',
       'Potassium_(mg)', 'Zinc_(mg)', 'Copper_mg)',
       'Manganese_(mg)', 'Selenium_(µg)', 'Vit_C_(mg)', 'Thiamin_(mg)',
       'Riboflavin_(mg)', 'Niacin_(mg)', 'Panto_Acid_mg)', 'Vit_B6_(mg)',
       'Folate_Tot_(µg)', 'Folic_Acid_(µg)', 'Food_Folate_(µg)',
       'Folate_DFE_(µg)', 'Choline_Tot_ (mg)', 'Vit_B12_(µg)', 'Vit_A_IU',
       'Vit_A_RAE', 'Retinol_(µg)', 'Alpha_Carot_(µg)', 'Beta_Carot_(µg)',
       'Beta_Crypt_(µg)', 'Lycopene_(µg)', 'Lut+Zea_ (µg)', 'Vit_E_(mg)',
       'Vit_D_µg', 'Vit_D_IU', 'Vit_K_(µg)', 'FA_Sat_(g)', 'FA_Mono_(g)',
       'FA_Poly_(g)', 'Refuse_Pct'], inplace=True)

In [7]:
usda.head()

,Energ_Kcal,Protein_(g),Sugar_Tot_(g),Sodium_(mg),Cholestrl_(mg),GmWt_1,GmWt_Desc1,GmWt_2,GmWt_Desc2,desc
id,,,,,,,,,,
1001,717.0,0.85,0.06,643.0,215.0,5.00,"1 pat, (1"" sq, 1/3"" high)",14.2,1 tbsp,butter with salt
1002,718.0,0.49,0.06,583.0,225.0,3.80,"1 pat, (1"" sq, 1/3"" high)",9.4,1 tbsp,butter whipped with salt
1003,876.0,0.28,0.00,2.0,256.0,12.80,1 tbsp,205.0,1 cup,butter oil anhydrous
1004,353.0,21.40,0.50,1146.0,75.0,28.35,1 oz,17.0,1 cubic inch,blue cheese
1005,371.0,23.24,0.51,560.0,94.0,132.00,"1 cup, diced",113.0,"1 cup, shredded",brick cheese


Rename columns in usda

In [8]:
usda.rename(columns={'Energ_Kcal': 'cal', 'GmWt_1': 'hwt_1', 'GmWt_2': 'hwt_2', 'Protein_(g)': 'protein', 'Sugar_Tot_(g)': 'sugar', 'Sodium_(mg)': 'sodium', 'Cholestrl_(mg)': 'cholesterol', 'GmWt_Desc1' : 'house_1', 'GmWt_Desc2': 'house_2'}, inplace=True)

Make recipes into a dataframe

In [9]:
cols = {}
keys = []
colnames = ['title', 'instructions', 'picture_link', 'src']
for col in colnames:
    cols[col] = []
# iterate over recipes
for rid, recipe in recipes.items():
    keys.append(rid)
    for col in colnames:
        if col not in recipe or not recipe[col]:
            cols[col].append(np.NaN)
        else:
            cols[col].append(recipe[col])
# init df
recipes_df = pd.DataFrame(index=keys, data=cols)
recipes_df.head(200)
recipes_df['title'].unique()

array(['Slow Cooker Chicken and Dumplings',
       'Awesome Slow Cooker Pot Roast', 'Brown Sugar Meatloaf', ...,
       'Fresh Cheese with Spinach ',
       'Jalapeño and Lime–Marinated Skirt Steak Tacos ',
       'Semolina–Lemon Syrup Cakes '], dtype=object)

In [10]:
ing_in.columns

Index(['comment', 'display', 'unit', 'name', 'input', 'qty', 'other',
       'usda_id'],
      dtype='object')

In [11]:
# add row to usda for missing ingredient
missing = pd.DataFrame(np.NaN, index=[-1], columns=usda.columns.values)
missing.loc[1, 'desc'] = 'NOMATCH'
usda_new = usda.append(missing, verify_integrity=True)
# remove name from usda and rename desc
usda.rename(columns={'desc': 'name'}, inplace=True)

Check primary key constraints

In [12]:
# verify that ids for usda_new, ing_in, and recipes are unique 
print(usda_new.index.duplicated().any())
print(recipes_df.index.duplicated().any())
print(ing_in.index.duplicated().any())
usda_new.rename(columns={'desc':'name'}, inplace=True)
usda_new.index.name = 'usda_id'

False
False
False


Check foreign key constraints

In [13]:
print(ing_in['usda_id'].isin(usda_new.index.values).all())
print(ing_in.index.get_level_values('rid').isin(recipes_df.index.values).all())

True
True


In [14]:
a= [2, 4]
sum(a)
Fraction('1')

Fraction(1, 1)

In [15]:
def convert(x, regexp=None):
    if not isinstance(x, str) and math.isnan(x) or x == 'nan':
        return np.NaN
    if regexp.search(x):
        return np.NaN
    if '-' in x:
        x = x.split('-')[0]
    if '–' in x:
        x = x.split('–')[0]
    if '—' in x:
        x = x.split('—')[0]
    xlist = x.split(' ')
    xlist = [float(Fraction(el)) for el in xlist]
    return sum(xlist)
# column type conversions
org = ing_in.loc[:, 'qty']
# replacements
org = org.str.replace('*', '')
org = org.str.replace('"', '')
org = org.str.replace(',', '')
org = org.str.replace(';', '')
org = org.str.replace('¼', '1/4')
org = org.str.replace('‟', '')
org = org.str.replace('½', '1/2')
org = org.str.replace('¾', '3/4')
org = org.str.replace('§', '')

org = org.apply(convert, regexp=re.compile(r'[A-Za-z]'))
ing_in.loc[org.index, 'qty'] = org.values

In [75]:
ing_in['qty'] = ing_in['qty'].astype(np.float)

In [92]:
# column and index names for each dataframe
print("usda")
print(usda_new.columns)
print(usda_new.dtypes)
print(usda_new.index.name)
print(usda_new.head())

print("ing_in")
print(ing_in.columns)
print(ing_in.dtypes)
print(ing_in.index.names)
print(ing_in.head())

print("recipes")
print(recipes_df.columns)
print(recipes_df.index.name)
print(recipes_df.head())

usda
Index(['cal', 'protein', 'sugar', 'sodium', 'cholesterol', 'hwt_1', 'house_1',
       'hwt_2', 'house_2', 'name'],
      dtype='object')
cal            float64
protein        float64
sugar          float64
sodium         float64
cholesterol    float64
hwt_1          float64
house_1         object
hwt_2          float64
house_2         object
name            object
dtype: object
usda_id
           cal  protein  sugar  sodium  cholesterol   hwt_1  \
usda_id                                                       
1001     717.0     0.85   0.06   643.0        215.0    5.00   
1002     718.0     0.49   0.06   583.0        225.0    3.80   
1003     876.0     0.28   0.00     2.0        256.0   12.80   
1004     353.0    21.40   0.50  1146.0         75.0   28.35   
1005     371.0    23.24   0.51   560.0         94.0  132.00   

                            house_1  hwt_2          house_2  \
usda_id                                                       
1001     1 pat,  (1" sq, 1/3" high)   

In [94]:
ing_in.index.levels[1].dtype

dtype('int64')

In [1]:
# print maximum length of string in each column
# recipes
# largedata clob required for instructions
for col in ['title', 'instructions', 'picture_link', 'src']:
    print('%s : %d' % (col, np.amax(recipes_df[col].str.len())))
print('rid: %d' % np.amax(recipes_df.index.str.len().values))

print('')

for col in ['comment', 'display', 'unit', 'name', 'input', 'other']:
    print('%s : %d' % (col, np.amax(ing_in[col].str.len())))

print(' ')

for col in ['house_1', 'house_2', 'name']:
    print('%s : %d' % (col, np.amax(usda_new[col].str.len())))


NameError: name 'np' is not defined

## Save outputs

In [32]:
def save(path, obj):
    obj.to_excel(path, na_rep='', header=True, index=True)
    pass
ing_in_output = 'data/join/ing_in'
big_save(ing_in_output, ing_in, 30)

In [33]:
def big_save(path, obj, num):
    size = int(obj.shape[0] / num)
    counter = 0
    for start in range(0, obj.shape[0], size):
        subset = obj.iloc[start:start + size]
        save('%s_%d.xlsx' % (path, counter), subset)
        counter += 1

In [34]:
ing_in.columns
ing_in.rename(columns={'comment':'comm'}, inplace=True)